<a href="https://colab.research.google.com/github/nlovoldegar/nlovoldegar_london_hotels/blob/main/%5BNicole_Lovold_Egar%5DAssignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nicolelovoldegar","key":"6f3b82ca462a786dec38b73cb86062fc"}'}

In [ ]:
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                             title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                       COVID-19 Dataset                                      5MB  2022-11-13 15:47:17           5806        174  1.0              
madhurpant/world-deaths-and-causes-1990-2019                    World Deaths and Causes (1990 - 2019)               442KB  2022-11-29 07:09:27            958         23  1.0              
thedevastator/jobs-dataset-from-glassdoor                       Salary Prediction                                     3MB  2022-11-16 13:52:31           3847         89  1.0              
thedevastator/how-much-sleep-do-americans-really-get        

In [ ]:
!kaggle datasets download -d hamzafarooq50/hotel-listings-and-reviews/HotelListInLondon__en2019100120191005.csv

 59% 5.00M/8.54M [00:00<00:00, 44.2MB/s]
100% 8.54M/8.54M [00:00<00:00, 62.9MB/s]


In [ ]:
!ls

hotel-listings-and-reviews.zip	kaggle.json  sample_data


In [ ]:
import re

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
import os
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy import displacy

In [ ]:
text = """this is a base text"""

doc = nlp(text)
sentence_spans = list(doc.sents)
displacy.render(doc, jupyter = True, style="ent")

/usr/local/lib/python3.8/dist-packages/spacy/displacy/__init__.py:206: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [ ]:
stopwords = list(STOP_WORDS)
from string import punctuation
punctuation = punctuation+ '\n'

In [ ]:
import pandas as pd

import scipy.spatial
import pickle as pkl

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.9 MB/s 
     |████████████████████████████████| 5.8 MB 16.9 MB/s 
     |████████████████████████████████| 1.3 MB 52.1 MB/s 
     |████████████████████████████████| 182 kB 56.9 MB/s 
     |████████████████████████████████| 7.6 MB 42.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=67599c02c8a0976f237ed890a610006fcabc01451c383ce7ed9b75acde1d6c78
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
#embedder = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
df = pd.read_csv("/content/HotelListInLondon__en2019100120191005.csv",sep=",", encoding='cp1252')

In [ ]:
!kaggle datasets download -d hamzafarooq50/hotel-listings-and-reviews

hotel-listings-and-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
df.head()

,Unnamed: 0,hotel_name,url,locality,reviews,tripadvisor_rating,checkIn,checkOut,price_per_night,booking_provider,no_of_deals,hotel_features
0,0,Travelodge London City hotel,http://www.tripadvisor.com/Hotel_Review-g18633...,London,1035,NaN,10/1/2019,10/5/2019,NaN,Expedia.com,8,Travelodge London City hotel is an excellent c...
1,1,Hampton by Hilton London Docklands,http://www.tripadvisor.com/Hotel_Review-g18633...,London,550,NaN,10/1/2019,10/5/2019,$159,Booking.com,14,Everyone needs a place to lay their weary head...
2,2,Sunborn London,http://www.tripadvisor.com/Hotel_Review-g18633...,London,3558,NaN,10/1/2019,10/5/2019,NaN,Expedia.com,13,Sunborn London is located at the western end o...
3,3,The Walrus Hostel,http://www.tripadvisor.com/Hotel_Review-g18633...,London,585,NaN,10/1/2019,10/5/2019,$58,Hostelworld.com,5,"A 5 minute walk from London Eye, Big Ben, Hous..."
4,4,"Point A Hotel, London Kings Cross St Pancras",http://www.tripadvisor.com/Hotel_Review-g18633...,London,6295,NaN,10/1/2019,10/5/2019,NaN,Expedia.com,14,Kings Cross has become a must-visit destinatio...


In [ ]:
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

In [ ]:
df['hotel_name'].value_counts()

Travelodge London City hotel                      1
Hampton by Hilton London Docklands                1
hub by Premier Inn London Westminster Abbey       1
Premier Inn London Hanger Lane Hotel              1
Lansbury Heritage Hotel                           1
Kensington Gardens Hotel                          1
The Hide London                                   1
Pelican London Hotel and Residence                1
The Captain Cook                                  1
The Lion & Key Hotel                              1
DoubleTree by Hilton London Greenwich             1
The Mad Hatter Hotel                              1
Brit Hotels London Court                          1
Ibis London Shepherds Bush - Hammersmith Hotel    1
Hotel 55                                          1
The Z Hotel City                                  1
Ibis London Canning Town                          1
Park Plaza London Park Royal                      1
Holiday Inn London - West                         1
Charlotte Gu

In [ ]:
df['hotel_name'].drop_duplicates()

0                       Travelodge London City hotel
1                 Hampton by Hilton London Docklands
2                                     Sunborn London
3                                  The Walrus Hostel
4       Point A Hotel, London Kings Cross St Pancras
5                                   St. Athans Hotel
6                                 The W14 Kensington
7                       Z Hotel Tottenham Court Road
8                               The Westbridge Hotel
9                               Novotel London Excel
10                                     Montana Hotel
11                             Charlotte Guest House
12                         Holiday Inn London - West
13                      Park Plaza London Park Royal
14                          Ibis London Canning Town
15                                  The Z Hotel City
16                                          Hotel 55
17    Ibis London Shepherds Bush - Hammersmith Hotel
18                          Brit Hotels London

In [ ]:
df_combined = df.sort_values(['hotel_name']).groupby('hotel_name', sort=False).hotel_features.apply(''.join).reset_index(name='hotel_features')


In [ ]:
df.iloc[6].hotel_features

'The W14 Hotel London is a 3 Star Hotel located in trendy Kensington, London. The hotel spans four beautiful Victorian town houses with original features and has been given a clean, contemporary look. Situated in a quiet, leafy, residential road close to the prestigious shops of High Street Kensington, the W14 Hotel London is just a 10 minute underground ride from the sights and sounds of Central London. You can view the W14 Hotel London on our interactive map. Please also read the W14 Hotel London Reviews.'

In [ ]:
df_combined.head().T

,0,1,2,3,4
hotel_name,Brit Hotels London Court,Charlotte Guest House,DoubleTree by Hilton London Greenwich,Hampton by Hilton London Docklands,Holiday Inn London - West
hotel_features,London Court Hotel boasts 21 tastefully furnis...,"Within a quiet residential area of London, the...","Located in Royal Greenwich, DoubleTree By Hilt...",Everyone needs a place to lay their weary head...,Welcome to Holiday Inn London-West. Discover a...


In [ ]:
import re

df_combined['hotel_features'] = df_combined['hotel_features'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['hotel_features']= df_combined['hotel_features'].apply(lambda x: lower_case(x))


In [ ]:
df = df_combined

In [ ]:
df_sentences = df_combined.set_index("hotel_features")
df_sentences = df_sentences["hotel_name"].to_dict()
df_sentences_list = list(df_sentences.keys())
len(df_sentences_list)

30

In [ ]:
list(df_sentences.keys())[:5]

['london court hotel boasts 21 tastefully furnished stylish en suite bedrooms which are fully equipped with all the amenities making them an ideal choice for accommodation for leisure and business travelers we at the london court hotel are proud of our helpful and knowledgeable staff who are always at hand should you need advice on where to go and what to do within easy reach of our small boutique hotel',
 'within a quiet residential area of london the traditional charlotte guest house is just a 5minute walk from the west hampstead tube and railway stations offering guest accommodation in 5 beautiful victorian buildings all within 2 minutes walk of the main reception each room includes a tv tea and coffee making facilities and free wifi some rooms benefit from their own private bathrooms while others share facilities hairdryers and ironing facilities are available on request we now have beautifully interior designed studio suites with contemporary bathrooms  see our web site for full d

In [ ]:
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

In [ ]:
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]

100%|██████████| 30/30 [00:00<00:00, 205939.64it/s]


In [ ]:
# Converts each combined review and creates an embedding for each hotel
corpus = df_sentences_list
corpus_embeddings = embedder.encode(corpus,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
corpus_embeddings[0].shape

(384,)

In [ ]:
queries = ['Hotel near St Pancras Station that have a spa']
query_embeddings = embedder.encode(queries,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import torch

# Query sentences:
queries = ['Hotel in london close to buckingham palace',
           'Hotel in london less than 200 per night']


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("(Score: {:.4f})".format(score))
        print(corpus[idx], "(Score: {:.4f})".format(score))
        row_dict = df.loc[df['hotel_features']== corpus[idx]]
        print("paper_id:  " , row_dict['hotel_name'] , "\n")
    # for idx, distance in results[0:closest_n]:
    #     print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
    #     print("Paragraph:   ", corpus[idx].strip(), "\n" )
    #     row_dict = df.loc[df['all_review']== corpus[idx]]
    #     print("paper_id:  " , row_dict['Hotel'] , "\n")







Query: Hotel in london close to buckingham palace

Top 5 most similar sentences in corpus:
(Score: 0.6629)
kensington gardens hotel is a 17 bedroom 2 star central london hotel located 5 minutes walk from hyde park bayswater and queensway underground stations are both within a few minutes walking distance the hotel is centrally situated for both the business executive and the holidaymaker with the west end and its shops and theatres within easy reach free wifi is always included when making a reservation with kensington gardens hotel checkin time is from 1 pm onward there is a complimentary luggage storage facility at the hotel reception is manned from 8 am to 5 pm the reception at phoenix hotel next door at 18 kensington gardens square will assist during other hours (Score: 0.6629)
paper_id:   8    Kensington Gardens Hotel
Name: hotel_name, dtype: object 

(Score: 0.6279)
welcome to holiday inn londonwest discover a fantastic arrival experience at the holiday inn london  west with 

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
embeddings = model.encode(corpus)
#print(embeddings)

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
# Query sentences:
queries = ['Hotel near Kings Cross and good restaurants'
           ]

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    
    query_embedding = model.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("(Score: {:.4f})".format(score))
        print(corpus[idx], "(Score: {:.4f})".format(score))
        row_dict = df.loc[df['hotel_features']== corpus[idx]]
        print("paper_id:  " , row_dict['hotel_name'] , "\n")





Query: Hotel near Kings Cross and good restaurants

Top 5 most similar sentences in corpus:
(Score: 0.6192)
kings cross has become a mustvisit destination and your point a hotel london kings cross is right at the heart of it bars restaurants and of course kings cross and st pancras eurostar stations are all moments away (Score: 0.6192)
paper_id:   14    Point A Hotel, London Kings Cross St Pancras
Name: hotel_name, dtype: object 

(Score: 0.5412)
with excellent transport links park plaza london park royal is ideal for business travellers seeking efficiency and comfort the hotel is well connected to the rail network and londons international airports wembley stadium and the sse arena are on the doorstep the city and londons commercial areas are a short tube ride away and the hotel is close to the major motorways the hotels 212 guest rooms include 48inch smart led tvs walkin showers elemis bathroom amenities and complimentary highspeed wifi the restaurant serves breakfast lunch and d

In [ ]:
hits = util.semantic_search(query_embedding, embeddings, top_k=5)
hits = hits[0]      #Get the hits for the first query
for hit in hits:
  print (hit)
  print("(Score: {:.4f})".format(hit['score']))
  print(corpus[hit['corpus_id']])
  row_dict = df.loc[df['hotel_features']== corpus[hit['corpus_id']]]
  print("paper_id:  " , row_dict['hotel_name'] , "\n")

{'corpus_id': 14, 'score': 0.6191765069961548}
(Score: 0.6192)
kings cross has become a mustvisit destination and your point a hotel london kings cross is right at the heart of it bars restaurants and of course kings cross and st pancras eurostar stations are all moments away
paper_id:   14    Point A Hotel, London Kings Cross St Pancras
Name: hotel_name, dtype: object 

{'corpus_id': 12, 'score': 0.5412082672119141}
(Score: 0.5412)
with excellent transport links park plaza london park royal is ideal for business travellers seeking efficiency and comfort the hotel is well connected to the rail network and londons international airports wembley stadium and the sse arena are on the doorstep the city and londons commercial areas are a short tube ride away and the hotel is close to the major motorways the hotels 212 guest rooms include 48inch smart led tvs walkin showers elemis bathroom amenities and complimentary highspeed wifi the restaurant serves breakfast lunch and dinner while the ba

In [ ]:
query_embedding.shape

torch.Size([768])

In [ ]:
df

,hotel_name,hotel_features
0,Brit Hotels London Court,london court hotel boasts 21 tastefully furnis...
1,Charlotte Guest House,within a quiet residential area of london the ...
2,DoubleTree by Hilton London Greenwich,located in royal greenwich doubletree by hilto...
3,Hampton by Hilton London Docklands,everyone needs a place to lay their weary head...
4,Holiday Inn London - West,welcome to holiday inn londonwest discover a f...
5,Hotel 55,hotel 55 is a 29 room 4 star boutique hotel ju...
6,Ibis London Canning Town,awake refreshed from the best nights sleep eve...
7,Ibis London Shepherds Bush - Hammersmith Hotel,we are now open and ready to welcome you in ou...
8,Kensington Gardens Hotel,kensington gardens hotel is a 17 bedroom 2 sta...
9,Lansbury Heritage Hotel,canary wharfs only heritage hotel welcome to t...


End

In [ ]:
!pip install -q streamlit

     |████████████████████████████████| 9.2 MB 5.2 MB/s 
     |████████████████████████████████| 237 kB 52.8 MB/s 
     |████████████████████████████████| 78 kB 7.6 MB/s 
     |████████████████████████████████| 4.7 MB 56.9 MB/s 
     |████████████████████████████████| 182 kB 55.1 MB/s 
     |████████████████████████████████| 164 kB 70.1 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 
     |████████████████████████████████| 51 kB 8.0 MB/s 
